In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib widget

from pathlib import Path
import typing as t

import numpy
from matplotlib import pyplot

In [2]:
import json

from fft_bench.main import BenchResult

nb_path = Path("").resolve()

paths = [
    Path('bench_supercloud.json'),
]

results: t.List[BenchResult] = []

for path in paths:
    with open(path, 'r') as f:
        for line in f:
            results.append(BenchResult(**json.loads(line)))

In [3]:
results

In [6]:
from itertools import chain

numpy_results = list(filter(lambda r: r.backend == 'numpy', results))
#accel_results = list(filter(lambda r: r.backend == 'accelerate', results))
cuda_results = list(filter(lambda r: r.backend == 'cuda', results))

fig, ax = pyplot.subplots()

ax.set_yscale('log')
ax.set_xscale('log')

for backend in ('numpy', 'cuda'):
    for double in (False, True):
        rs = list(filter(lambda r: r.backend == backend and r.double == double and r.n_d == 2, results))

        xs = list(chain.from_iterable(map(lambda r: (2**(r.log2n),) * len(r.times), rs)))
        ys = list(chain.from_iterable(map(lambda r: r.times, rs)))
        ax.plot(xs, ys, ' ', marker='o' if double else 'x', color='blue' if backend == 'numpy' else 'red', markersize=5,
                label=f"{backend.title()} {'Double' if double else 'Single'}")

n_powers_x = 14
n_powers_y = 16
ax.set_xlim(32, 32 * 2**n_powers_x)
ax.set_ylim(2e-5, 2e-5 * 2**n_powers_y)
n_powers = max(n_powers_y, n_powers_x)
ax.plot([32, 32 * 2**n_powers], [2e-5, 2e-5 * 2**n_powers])
ax.set_aspect(n_powers_y / n_powers_x)

ax.legend()
ax.set_xlabel('# Points')
ax.set_ylabel('Time per slice (s)')

pyplot.show()

In [8]:
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize

fig, ax = pyplot.subplots()

ax.set_yscale('log')
ax.set_xscale('log')

slices_cmap = get_cmap('hot')
slices_norm = Normalize(vmin=1, vmax=8)

for double in (False, True):
    rs = list(filter(lambda r: r.backend == 'cuda' and r.double == double and r.n_d == 2, results))

    for n_slices in set(map(lambda r: r.n_slices_per_call, rs)):
        n_slices_rs = list(filter(lambda r: r.n_slices_per_call == n_slices, rs))
        xs = list(chain.from_iterable(map(lambda r: (2**(r.log2n),) * len(r.times), n_slices_rs)))
        ys = list(chain.from_iterable(map(lambda r: r.times, n_slices_rs)))
        ax.plot(xs, ys, ' ', marker='o' if double else 'x', color=slices_cmap(slices_norm(n_slices)), markersize=5,
                label=f"{backend.title()} {'Double' if double else 'Single'} {n_slices} slices")

n_powers_x = 14
n_powers_y = 16
ax.set_xlim(32, 32 * 2**n_powers_x)
ax.set_ylim(2e-5, 2e-5 * 2**n_powers_y)
n_powers = max(n_powers_y, n_powers_x)
ax.plot([32, 32 * 2**n_powers], [2e-5, 2e-5 * 2**n_powers])
ax.set_aspect(n_powers_y / n_powers_x)

ax.legend()
ax.set_xlabel('# Points')
ax.set_ylabel('Time per slice (s)')

pyplot.show()